In [1]:
from router import router_api
router_api('bulk')

In [2]:
import logging

from langchain_openai import ChatOpenAI
from sisyphus.chain import Paragraph
from sisyphus.heas.extract_lc import extract_bulk
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import Filter, Writer, run_chains_with_extarction_history_multi_threads

logger = logging.getLogger(__name__)
logger.setLevel(10)
fh = logging.FileHandler('heas_bulk.log')
fh.setLevel(10)
ft = logging.Formatter('%(message)s')
fh.setFormatter(ft)
logger.addHandler(fh)

model = ChatOpenAI(temperature=0, model='gpt-4.1', max_tokens=15000)

/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
database_name = 'labeled'
result_db_name = 'heas_bulk'
target_dir = 'TEST_FILES'

In [4]:
db = get_plain_articledb(database_name)
getter = Filter(db)
result_db = get_create_resultdb(result_db_name)
writer = Writer(result_db)
 
def load(docs):
    return [Paragraph.from_labeled_document(doc, id_) for id_, doc in enumerate(docs)]

from functools import partial
extract_p = partial(extract_bulk, extraction_model=model)
chain = getter + load + extract_p + writer

/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/pydantic/main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [5]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pydantic') # avoide the case that we convert json string to python object will trigger pydantic warning

In [6]:
run_chains_with_extarction_history_multi_threads(chain, target_dir, 5, result_db_name)

ValueError: no file needed to be extracted